In [1]:
import synonyms

Building prefix dict from /data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/vocab.txt ...
Loading model from cache /tmp/jieba.u1ee567e3d900769013ee0717e970f060.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [/data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/vocab.txt] ... 


Loading model cost 1.215 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [/data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/stopwords.txt] ...
>> Synonyms on loading vectors [/data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/words.vector.gz] ...


In [2]:
synonyms.nearby("爸爸")

(['爸爸', '妈妈', '爷爷', '奶奶', '老婆', '姐姐', '爸妈', '老公', '老爸', '外婆'],
 [1.0,
  0.9197904,
  0.8369896,
  0.83030784,
  0.80404544,
  0.79813045,
  0.7979228,
  0.7714372,
  0.77116084,
  0.7584483])

In [4]:
stop_words = {word.strip() for word in open('data/baidu_stopwords.txt', 'r', encoding='utf-8').readlines()}

In [13]:
# 同义词替换(SR): 从句子中随机选择n个不是停用词的词 用随机选择的同义词之一替换这些单词中的每一个
import random

def get_synonyms(word):
    sys = set(synonyms.nearby(word)[0])
    if word in sys:
        sys.remove(word)
        
    return list(sys)

def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced > n:
            break
            
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')
    
    return new_words

In [6]:
# 随机删除(RD): 以概率p随机删除句子中的每个单词

def random_deletion(words, p):
    if len(words) == 1:
        return words
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
            
    if len(new_words) == 0:
        random_int = random.randint(0, len(words) - 1)
        return [words[random_int]]
    
    return new_words

In [7]:
# 随机交换(RS): 随机选择句子中的两个单词并交换他们的位置 做n次
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    
    cnt = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        cnt += 1
        if cnt > 3:
            return new_words
        
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    
    return new_words

In [8]:
# 随即插入(RI): 在句子中随机找到一个词, 并找出其同义词, 且该词不是停用词, 将该同义词插入句子的随机位置 做n次
def add_word(new_words):
    synonyms = []
    cnt = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words) - 1)]
        synonyms = get_synonyms(random_word)
        cnt += 1
        if cnt >= 10:
            return
    
def random_insert(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
        
    return new_words

In [10]:
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    words = synonyms.seg(sentence)[0]
    num_words = len(words)
    augmented_sentence = []
    num_new_per_technique = int(num_aug / 4) + 1
    
    n_sr = max(1, (int)(alpha_sr * num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))
    
    # sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentence.append(' '.join(a_words))
    # ri
    for _ in range(num_new_per_technique):
        a_words = random_insert(words, n_ri)
        augmented_sentence.append(' '.join(a_words))
    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentence.append(' '.join(a_words))
    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentence.append(' '. join(a_words))
        
    random.shuffle(augmented_sentence)
    if num_aug >= 1:
        augmented_sentence = augmented_sentence[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentence)
        augmented_sentence = [s for s in augmented_sentence if random.uniform(0, 1) < keep_prob]
    
    return augmented_sentence

In [11]:
words = synonyms.seg('目前华为部分型号的手机产品出现货少的现象')
print(words)

(['目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '货', '少', '的', '现象'], ['t', 'nr', 'n', 'n', 'uj', 'n', 'n', 'v', 'n', 'a', 'uj', 'n'])


In [14]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

['9 月 15 日 ， ， 台积电 下去 高通 、 三星 等 华为 的 重要 合作伙伴 ， 的 没有 美国 的 美国 许可证 芯片 都 无法 供应 ， 给 华为 ， 而 中芯国际 等 遭受 芯片 企业 ， 也 因 采用 相关 技术 以来 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 出现 产品 货 少 只要 现象 ， 若 该 形势 持续 、 ， 华为 手机 业务 将 国产 重创 。',
 '9 月 15 日 技术 的 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 目前 ， 都 无法 供应 持续 给 华为 ， 而 中芯国际 等 国产 芯片 企业 货 也 因 该 美国 以来 ， 而 无法 供货 给 华为 。 许可证 现象 部分 型号 的 手机 产品 出现 ， 少 ， 华为 ， 若 采用 形势 芯片 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 日 ， 、 高通 三星 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 企业 ， 也 因 采用 美国 技术 ， 而 供货 给 华为 。 目前 华为 部分 型号 的 产品 出现 货 少 的 现象 ， 若 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 日 15 以来 等 台积电 、 高通 手机 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 部分 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 、 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 少 无法 供货 给 华为 。 目前 华为 的 型号 的 ， 产品 出现 货 而 的 现象 ， 若 遭受 形势 持续 下去 ， 华为 手机 业务 将 该 重创 。',
 '9 月 15 日 以来 ， 、 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 的 相关 许可证 ， 都 无法 供应 芯片 给 ， 而 中芯国际 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 产品 出现 货 少 的 现象 ， 若 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 

#### 闭包数据增强
数据集中每条数据有两个句子\
a和b相似 a和c相似\
则b和c相似\
a和d不相似 0\
则b和d不相似

In [15]:
import pandas as pd
from tqdm.notebook import tqdm
import json

def parse_data(path, data_type='train'):
    sentence_a, sentence_b, labels = [], [], []
    with open(path, 'r', encoding='utf-8') as f:
        for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
            line = json.loads(line)
            sentence_a.append(line['sentence1'])
            sentence_b.append(line['sentence2'])
            if data_type != 'test':
                labels.append(int(line['label']))
            else:
                labels.append(0)
    df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns=['text_a', 'text_b', 'labels'])
    
    return df

In [16]:
train_df = parse_data('data/train.json', data_type='train')

Reading train data:   0%|          | 0/34334 [00:00<?, ?it/s]

In [17]:
train_df.head(3)

,text_a,text_b,labels
0,蚂蚁借呗等额还款可以换成先息后本吗,借呗有先息到期还本吗,0
1,蚂蚁花呗说我违约一次,蚂蚁花呗违约行为是什么,0
2,帮我看一下本月花呗账单有没有结清,下月花呗账单,0


In [35]:
for data in train_df.groupby(by=['text_a']):
    if len(data[1]) == 3:
        print(data[0])
        print(type(data[1]))
        print(data[1])
        break

为何我的花呗不能用
<class 'pandas.core.frame.DataFrame'>
          text_a       text_b  labels
13425  为何我的花呗不能用     为什么花呗用不了       1
14005  为何我的花呗不能用    花呗不能正常使用了       1
15175  为何我的花呗不能用  我现在是不是用不了花呗       0


In [36]:
from collections import defaultdict

def aug_group_by_a(df):
    aug_data = defaultdict(list)
    # 以text_a句子为a
    for g, data in df.groupby(by=['text_a']):
        if len(data) < 2:
            continue
        for i in range(len(data)):
            for j in range(i + 1, len(data)):
                # 取出b的值 a, b的label
                row_i_text = data.iloc[i, 1]
                row_i_label = data.iloc[i, 2]
                # 取出c的值 a, c的label
                row_j_text = data.iloc[j, 1]
                row_j_label = data.iloc[j, 2]
                
                if row_i_label == row_j_label == 0:
                    continue
                aug_label = 1 if row_i_label == row_j_label == 1 else 0
                
                aug_data['text_a'].append(row_i_text)
                aug_data['text_b'].append(row_j_text)
                aug_data['labels'].append(aug_label)
    
    return pd.DataFrame(aug_data)

In [38]:
aug_train_a = aug_group_by_a(train_df)
print(len(aug_train_a))

228


In [39]:
from extra_file.bucket_sampler import SortedSampler, BucketBatchSampler
from extra_file.EMA import *

In [48]:
import random
import numpy as np
import torch

config = {
        'train_file_path': 'data/train.json',
        'dev_file_path': 'data/dev.json',
        'test_file_path': 'data/test.json',
        'output_path': '.',
        'model_path': '../NLP_Project/dataset/BERT_model/',
        'batch_size': 16,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 400,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2021
}

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2021

In [41]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [43]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append(inputs_dict['input_ids'])
    inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
    inputs['attention_mask'].append(inputs_dict['attention_mask'])
    inputs['labels'].append(label)

In [44]:
# 对偶数据增强
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
                     
    rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
    inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'],rl_inputs_dict['token_type_ids']))
    inputs['attention_mask'].append((lr_inputs_dict['attention_mask'],rl_inputs_dict['attention_mask']))
    inputs['labels'].append(label)

In [50]:
def read_data(config, tokenizer):
    train_df = parse_data(config['train_file_path'], data_type='train')
    dev_df = parse_data(config['dev_file_path'], data_type='dev')
    test_df = parse_data(config['test_file_path'], data_type='test')
    
    data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
    processed_data = {}
    unsup_data = defaultdict(list)
    
    for data_type, df in data_df.items():
        inputs = defaultdict(list)
        if data_type == 'train':
            reversed_inputs = defaultdict(list)
        
        for i, row in tqdm(df.iterrows(), desc=f'Processing {data_type} data', total=len(df)):
            label = 0 if data_type == 'test' else row[2]
            sentence_a, sentence_b = row[0], row[1]
            build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)
            
            if data_type.startswith("test"):
                build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)
            
            build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)
        processed_data[data_type] = inputs
        
    processed_data['unsup_data'] = unsup_data
    
    return processed_data

In [51]:
data = read_data(config, tokenizer)

Reading train data:   0%|          | 0/34334 [00:00<?, ?it/s]

Reading dev data:   0%|          | 0/4316 [00:00<?, ?it/s]

Reading test data:   0%|          | 0/3861 [00:00<?, ?it/s]

Processing train data:   0%|          | 0/34334 [00:00<?, ?it/s]

Processing dev data:   0%|          | 0/4316 [00:00<?, ?it/s]

Processing test data:   0%|          | 0/3861 [00:00<?, ?it/s]

In [52]:
from torch.utils.data import Dataset

class AFQMCDataset(Dataset):
    def __init__(self, data_dict):
        super(AFQMCDataset, self).__init__()
        self.data_dict = data_dict
        
    def __getitem__(self, idx):
        data = (self.data_dict['input_ids'][idx], self.data_dict['token_type_ids'][idx],
                self.data_dict['attention_mask'][idx], self.data_dict['labels'][idx])
        return data
    
    def __len__(self):
        return len(self.data_dict['input_ids'])

In [53]:
class Collator:
    def __init__(self, max_seq_len, tokenizer):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
        
    def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
        input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype=torch.long)
        token_type_ids = torch.zeros_like(input_ids)
        attention_mask = torch.zeros_like(input_ids)
        
        for i in range(len(input_ids_list)):
            seq_len = len(input_ids_list[i])
            if seq_len <= max_seq_len:
                input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype=torch.long)
                token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
                attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype=torch.long)
            else:
                input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len-1] + [self.tokenizer.sep_token_id], dtype=torch.long)
                token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype=torch.long)
                attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype=torch.long)
        
        labels = torch.tensor(labels_list, dtype=torch.long)
        
        return input_ids, token_type_ids, attention_mask, labels
        
    def __call__(self, examples):
        input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
        cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)
        
        input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)
        data_dict = {
            'input_ids': input_ids,
            'token_type_ids': token_type_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
        return data_dict

In [55]:
collate_fn = Collator(config['max_seq_len'], tokenizer)